# 投资组合收益率的因子模型

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import time
import os
import quiz_helper
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

### 数据包

In [ ]:
import os
import quiz_helper
from zipline.data import bundles

In [ ]:
os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '..', '..','data','module_4_quizzes_eod')
ingest_func = bundles.csvdir.csvdir_equities(['daily'], quiz_helper.EOD_BUNDLE_NAME)
bundles.register(quiz_helper.EOD_BUNDLE_NAME, ingest_func)
print('Data Registered')

### 构建管道引擎

In [ ]:
from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume
from zipline.utils.calendars import get_calendar

universe = AverageDollarVolume(window_length=120).top(500) 
trading_calendar = get_calendar('NYSE') 
bundle_data = bundles.load(quiz_helper.EOD_BUNDLE_NAME)
engine = quiz_helper.build_pipeline_engine(bundle_data, trading_calendar)

### 查看数据

构建管道引擎后，我们获取时段结束时股票池中的股票。我们将使用这些 ticker 生成风险模型的收益率数据。

In [ ]:
universe_end_date = pd.Timestamp('2016-01-05', tz='UTC')

universe_tickers = engine\
    .run_pipeline(
        Pipeline(screen=universe),
        universe_end_date,
        universe_end_date)\
    .index.get_level_values(1)\
    .values.tolist()
    
universe_tickers

In [ ]:
len(universe_tickers)

In [ ]:
from zipline.data.data_portal import DataPortal

data_portal = DataPortal(
    bundle_data.asset_finder,
    trading_calendar=trading_calendar,
    first_trading_day=bundle_data.equity_daily_bar_reader.first_trading_day,
    equity_minute_reader=None,
    equity_daily_reader=bundle_data.equity_daily_bar_reader,
    adjustment_reader=bundle_data.adjustment_reader)

## 获取股价数据的辅助函数

In [ ]:
from quiz_helper import get_pricing

## 将股价数据放入 dataframe 中

In [ ]:
returns_df = \
    get_pricing(
        data_portal,
        trading_calendar,
        universe_tickers,
        universe_end_date - pd.DateOffset(years=5),
        universe_end_date)\
    .pct_change()[1:].fillna(0) #convert prices into returns

returns_df

## 我们看看包含两支股票的投资组合

假设投资组合有两支股票，我们将使用 Apple 和 Microsoft 股票。

In [ ]:
aapl_col = returns_df.columns[3]
msft_col = returns_df.columns[312]
asset_return_1 = returns_df[aapl_col].rename('asset_return_aapl')
asset_return_2 = returns_df[msft_col].rename('asset_return_msft')
asset_return_df = pd.concat([asset_return_1,asset_return_2],axis=1)
asset_return_df.head(2)

## 因子收益率
我们通过对列表中的所有股票求平均值，创建一个虚拟“因子”。可以将其看做 490 支股票的相等加权指数，就好像能衡量市场的指标。再通过计算所有股票的中位数创建另一个虚拟因子。这么做主要是为了生成可以使用的数据。在后面的课程中，我们将介绍如何生成一些常见风险因子。

另请注意，我们将 axis 设为 1，从而计算每个时段（行）的值，而不是每列（资产）对应一个值。

In [ ]:
factor_return_1 = returns_df.mean(axis=1)
factor_return_2 = returns_df.median(axis=1)
factor_return_l = [factor_return_1, factor_return_2]

## 因子暴露度

因子暴露度是指股票对每个因子的暴露程度。稍后会详细讲解这个概念。暂时将其看做每支股票的每个因子对应一个数字。

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
"""
For now, just assume that we're calculating a number for each 
stock, for each factor, which represents how "exposed" each stock is
to each factor. 
We'll discuss how factor exposure is calculated later in the lessons.
"""
def get_factor_exposures(factor_return_l, asset_return):
    lr = LinearRegression()
    X = np.array(factor_return_l).T
    y = np.array(asset_return.values)
    lr.fit(X,y)
    return lr.coef_

In [ ]:
factor_exposure_l = []
for i in range(len(asset_return_df.columns)):
    factor_exposure_l.append(
        get_factor_exposures(factor_return_l,
                             asset_return_df[asset_return_df.columns[i]]
                            ))
    
factor_exposure_a = np.array(factor_exposure_l)

In [ ]:
print(f"factor_exposures for asset 1 {factor_exposure_a[0]}")
print(f"factor_exposures for asset 2 {factor_exposure_a[1]}")

## 小测验 1 投资组合的因子暴露度

暂时先直接设定投资组合权重，在后面的课程中，你将学习如何在投资组合优化过程中根据 alpha 因子和风险因子模型选择资产权重。

$\beta_{p,k} = \sum_{i=1}^{N}(x_i \times \beta_{i,k})$

In [ ]:
weight_1 = 0.60 #let's give AAPL a portfolio weight
weight_2 = 0.40 #give MSFT a portfolio weight
weight_a = np.array([weight_1, weight_2])

为了便于理解，请将每个值保存到单独的变量中，以便进行乘法和加法运算。检查你的投资组合因子暴露度计算结果是否与以下点积输出结果一致：

In [ ]:
weight_a.dot(factor_exposure_a)

In [ ]:
# TODO: calculate portfolio's exposure to factor 1
factor_exposure_1_1 = # ...
factor_exposure_2_1 = # ...
factor_exposure_p_1 = # ...
factor_exposure_p_1

In [ ]:
# TODO: calculate portfolio's exposure to factor 2
factor_exposure_1_2 = # ...
factor_exposure_2_2 = # ...
factor_exposure_p_2 = # ...
factor_exposure_p_2

## 小测验 2 计算投资组合收益率

为清楚起见，请将每个值存储到对应名称的变量中，并写出乘法和加法运算。

你可以检查你的答案是否与以下输出结果一致：

In [ ]:
asset_return_df.values.dot(weight_a)

In [ ]:
# TODO calculate the portfolio return
asset_return_1 = # ...
asset_return_2 = # ...
portfolio_return = # ...

portfolio_return = pd.Series(portfolio_return,index=asset_return_df.index).rename('portfolio_return')
portfolio_return.head(2)

## 小测验 3 因子的贡献

因子暴露度与因子收益率的乘积之和就是各个因子的贡献。也称为“公共收益率”。给定两个因子暴露度和两个因子收益率后，请计算投资组合的公共收益率。

In [ ]:
# TODO: Calculate the contribution of the two factors to the return of this example asset
common_return = # ...
common_return = common_return.rename('common_return')

common_return.head(2)

## 小测验 4 特有收益率
特有收益率是指投资组合收益率中不能由因子解释的部分。它等于实际收益率减去公共收益率。 
请计算股票的特有收益率。

In [ ]:
# TODO: calculate the specific return of this asset
specific_return = # ...
specific_return = specific_return.rename('specific_return')

## 可视化公共收益率和特有收益率

In [ ]:
return_components = pd.concat([common_return,specific_return],axis=1)
return_components.head(2)

In [ ]:
return_components.plot(title="asset return = common return + specific return");
pd.DataFrame(portfolio_return).plot(color='purple');

## 解答
[解答 notebook](factor_model_portfolio_return_solution.ipynb)